# There are two alternatives. 1. Parse final structures from MNDO trajectories to a Psi4 input file and run Psi4. After optimizing them I can use them as Torsiondrive initial structures. 2. just send un-optimized final structures to Torsiondrive as initial structures.
## 1 requires about 20 hours of running Psi4 and not necessarily the better way. So  I put the code of doing 1 at the end of notebook. The following is alternative 2.

It is helpful to generate more conformations by flipping the stator atoms. The trick is that I used consistent atom numbering at the very beginning of building the molecules. Current implementation works only for symmetric stator.

In [2]:


def final_frame_to_xyz_with_flip(natoms, path):
    %cd {path}
    %cd ..
    traj = path + "traj.out"
    finals = open("finalstructureswithflip.xyz", "a")
    nlines = natoms+2
    coords = !tail -{nlines} {traj}
    finals.write(coords.nlstr)
    finals.write("\n")

    for i in range(0,2): # The first two are header lines
        finals.write(coords[i])
        finals.write("\n")
    coords_noheader = coords[2:]
    for i in range(0,13): # Flip these stator heavy atoms that are symmetric around atom no. 7
        finals.write(coords_noheader[12-i])
        finals.write("\n")
    for i in range(13,28):
        finals.write(coords_noheader[i])
        finals.write("\n")
    for i in range(28,36): # Flip these stator hydrogen atoms that are symmetric around atom no. 32 and 33.
        finals.write(coords_noheader[63-i])
        finals.write("\n")
    for i in range(36,natoms):
        finals.write(coords_noheader[i])
        finals.write("\n")
    finals.close()
    return

def add_literature_structure(natoms, path, xyzfile):
    %cd {path}
    finals = open("finalstructureswithflip.xyz", "a")
    coords = !cat {xyzfile}
    finals.write(coords.nlstr)
    finals.write("\n")

    for i in range(0,2): # The first two are header lines
        finals.write(coords[i])
        finals.write("\n")
    coords_noheader = coords[2:]
    for i in range(0,13): # Flip these stator heavy atoms that are symmetric around atom no. 7
        finals.write(coords_noheader[12-i])
        finals.write("\n")
    for i in range(13,28):
        finals.write(coords_noheader[i])
        finals.write("\n")
    for i in range(28,36): # Flip these stator hydrogen atoms that are symmetric around atom no. 32 and 33.
        finals.write(coords_noheader[63-i])
        finals.write("\n")
    for i in range(36,natoms):
        finals.write(coords_noheader[i])
        finals.write("\n")
    finals.close()
    return   

add_literature_structure(48,"/home/winter/onsager/tsqm/dynamics/Ssampling/hop/","/home/winter/onsager/tsqm/Sm.xyz")
final_frame_to_xyz_with_flip(48,"/home/winter/onsager/tsqm/dynamics/Ssampling/hop/run0007/")
final_frame_to_xyz_with_flip(48,"/home/winter/onsager/tsqm/dynamics/Ssampling/hop/run0011/")
final_frame_to_xyz_with_flip(48,"/home/winter/onsager/tsqm/dynamics/Ssampling/hop/run0019/")
final_frame_to_xyz_with_flip(48,"/home/winter/onsager/tsqm/dynamics/Ssampling/hop/run0029/")
final_frame_to_xyz_with_flip(48,"/home/winter/onsager/tsqm/dynamics/Ssampling/hop/run0031_bk/")
final_frame_to_xyz_with_flip(48,"/home/winter/onsager/tsqm/dynamics/Ssampling/hop/run0034_bk/")
final_frame_to_xyz_with_flip(48,"/home/winter/onsager/tsqm/dynamics/Ssampling/hop/run0044_bk/")
final_frame_to_xyz_with_flip(48,"/home/winter/onsager/tsqm/dynamics/Ssampling/hop/run0050/")

/home/winter/onsager/tsqm/dynamics/Ssampling/hop
/home/winter/onsager/tsqm/dynamics/Ssampling/hop/run0007
/home/winter/onsager/tsqm/dynamics/Ssampling/hop
/home/winter/onsager/tsqm/dynamics/Ssampling/hop/run0011
/home/winter/onsager/tsqm/dynamics/Ssampling/hop
/home/winter/onsager/tsqm/dynamics/Ssampling/hop/run0019
/home/winter/onsager/tsqm/dynamics/Ssampling/hop
/home/winter/onsager/tsqm/dynamics/Ssampling/hop/run0029
/home/winter/onsager/tsqm/dynamics/Ssampling/hop
/home/winter/onsager/tsqm/dynamics/Ssampling/hop/run0031_bk
/home/winter/onsager/tsqm/dynamics/Ssampling/hop
/home/winter/onsager/tsqm/dynamics/Ssampling/hop/run0034_bk
/home/winter/onsager/tsqm/dynamics/Ssampling/hop
/home/winter/onsager/tsqm/dynamics/Ssampling/hop/run0044_bk
/home/winter/onsager/tsqm/dynamics/Ssampling/hop
/home/winter/onsager/tsqm/dynamics/Ssampling/hop/run0050
/home/winter/onsager/tsqm/dynamics/Ssampling/hop


# Below is alternative 1, which I haven't tested thoroughly

## `conda activate p4env` is needed for making the notebook run in psi4 environment.

In [8]:
def final_frame_to_psi4(natoms, path):
    traj = path + "traj.out"
    psi4input = open(path+"final.in", "a")
    psi4input.write("#! DFT optimization\nmemory 30 GB\nmolecule {\n")
    coords = !tail -{natoms} {traj}
    psi4input.write(coords.nlstr)
    psi4input.write("}\nset {\nbasis dzvp\nscf_type df\nreference uks\n}\noptimize('b3lyp-D3BJ')")
    psi4input.close()
    %cd {path}
    !psi4 -i final.in -o psi4.out -n 12 
    return


final_frame_to_psi4(48,"/home/winter/onsager/tsqm/dynamics/Osampling/hop/run0017/")

/home/winter/onsager/tsqm/dynamics/Osampling/hop/run0017
Optimizer: Optimization complete!


# Concatenate the output coords from finished Psi4 optimizations into a .xyz for Torsiondrive use.

/home/winter/onsager/tsqm/dynamics/Osampling/hop/run0017
